In [2]:
%load_ext autoreload
%autoreload 2

In [40]:
from torch.nn.functional import cross_entropy
import torch
import sys
from argparse import Namespace

In [4]:
sys.path.append('../')

In [5]:
import src.models.rnns

In [6]:
from src.models.rnns import GRU

In [7]:
from src.data import SARD

In [8]:
sard_dataset = SARD(Namespace())

In [9]:
sard_dataset.prepare_data()

PREPARING


In [10]:
sard_dataset.setup()

In [20]:
loader = sard_dataset.train_dataloader()

In [16]:
config_info = sard_dataset.config()

In [68]:
model = GRU(config_info)

In [69]:
model

GRU(
  (embed): Embedding(1235, 128)
  (rnn): GRU(128, 64, batch_first=True, bidirectional=True)
  (drop1): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=128, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=19, bias=True)
)

In [70]:
x, y = next(iter(loader))

In [71]:
x.shape

torch.Size([8, 1000])

In [72]:
y.shape

torch.Size([8, 19])

In [73]:
model(x).shape

torch.Size([8, 19])

In [74]:
e = model.embed

In [75]:
e

Embedding(1235, 128)

In [76]:
gru = model.rnn

In [77]:
gru

GRU(128, 64, batch_first=True, bidirectional=True)

In [78]:
t, h = gru(e(x))

In [79]:
t.shape, h.shape

(torch.Size([8, 1000, 128]), torch.Size([2, 8, 64]))

In [80]:
xh = torch.cat((h[-2, :], h[-1, :]), dim=-1)

In [81]:
xh

tensor([[-0.4607, -0.4282,  0.1003,  ..., -0.7474,  0.5012, -0.4603],
        [-0.5849, -0.4563, -0.2488,  ..., -0.7474,  0.5012, -0.4603],
        [-0.5323, -0.1876, -0.1156,  ..., -0.7474,  0.5010, -0.4603],
        ...,
        [ 0.4870, -0.0844,  0.1155,  ..., -0.7573,  0.5110, -0.4582],
        [-0.3649,  0.1806,  0.5250,  ..., -0.6255, -0.0860, -0.2704],
        [-0.1269, -0.5354,  0.1895,  ..., -0.7960,  0.4425, -0.4617]],
       grad_fn=<CatBackward0>)

In [82]:
xh.shape

torch.Size([8, 128])

In [83]:
model.forward(x).shape

torch.Size([8, 19])

In [84]:
model.fc1(xh).shape

torch.Size([8, 32])

In [85]:
model.fc2(model.fc1(xh)).shape

torch.Size([8, 19])